# 1. Preparation

In [ ]:
#@title Setup
#@markdown Run this code cell first. The button with the circle on the left will execute it.
import os
import subprocess

# Install required modules...
## R
!pip -q install rpy2==3.5.1
%load_ext rpy2.ipython
##
!curl -L -s -O https://github.com/YujiSue/ysngs/releases/download/ja/ysngs-colab.zip
!unzip -q ysngs-colab.zip
!pip install -q ./ysngs-main

# Environment construcion
from ysngs import Config, AppManager, WorkFlow, execCmd, common
cfg = Config()
cfg.setWorkSpace('/content')
cfg.setDefault()
cfg.makeDirs()
apps = AppManager(cfg)

## Folder Structure

Running the above will create the following folders:
- MyApp : Installation directory for analysis applications
- Sample : Upload directory for sequence data
- Script : Directory for other analysis scripts
- Reference : Storage directory for reference genomes and databases
- Preference : Directory for parameter and configuration files

# 2. Data Upload

Create a folder named "fastq" in your Google Drive and save your fastq (fq) or compressed (gz) files there.

In [ ]:
#@title Mount Google Drive
#@markdown This allows Colab to access Google Drive directly. Please follow the instructions that appear after running this cell.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Copy from Google Drive
#@markdown Running this will start the copy process. This assumes your data is in the "fastq" folder on Google Drive. If your data is elsewhere, please change the path below accordingly.

import glob
import os
drive_directory = '/content/drive/MyDrive/fastq' #@param {type:'string'}
res = common.execCmd(f'cp {drive_directory}/* /content/Sample', verbose=True)
samples = []
files = glob.glob('/content/Sample/*')
for file in files:
  if file.endswith('gz'):
    os.system('gunzip ' + os.path.join('/content/Sample/', file))
print('Completed.' if res[0] else 'Failed.')

# 3. Data Quality Check

In [ ]:
#@title Install Sequence Data Quality Analysis and Control Softwares
apps.install('fastqc')
apps.install('fastp')

In [ ]:
#@title Sequence Data Quality Analysis (Initial Check)

import glob
import os
import re
samples = []
paired_seq = True #@param {'type': 'boolean'}
files = glob.glob('/content/Sample/*')
for file in files:
  (dir, fname) = os.path.split(file)
  sample = fname[0:fname.rfind('_')]
  if sample not in samples:
    samples.append(sample)
  odir = os.path.join(f'/content/Output/{sample}/{os.path.splitext(fname)[0]}/primary')
  os.makedirs(odir, exist_ok=True)
  common.execCmd(f'/content/MyApp/FastQC/fastqc -o {odir} {file} &')


In [ ]:
#@title Adapter Cutting and Low-Quality Read Filtering
for sample in samples:
  files = glob.glob(f'/content/Sample/{sample}*')
  fqfiles = []
  for file in files:
    fqfiles.append(file)
  fqfiles.sort()
  if len(fqfiles) != 2:
    print(f'[Error] File count for "{sample}" is not 2.')
    continue

  os.chdir(f'/content/Output/{sample}')
  cmd = f'''/content/MyApp/fastp \
  -i {os.path.splitext(fqfiles[0])[0]}.cut.fq \
  -I {os.path.splitext(fqfiles[1])[0]}.cut.fq \
  -o {os.path.splitext(fqfiles[0])[0]}.filter.fq \
  -O {os.path.splitext(fqfiles[1])[0]}.filter.fq \
  -w 16'''
  common.execCmd(cmd, verbose=True)
  os.chdir('/content')

In [ ]:
#@title Sequence Data Quality Analysis (Post-Filter)
for sample in samples:
  files = glob.glob(f'/content/Sample/{sample}*.filter.*')
  for file in files:
    (dir, fname) = os.path.split(file)
    odir = f'/content/Output/{sample}/{os.path.splitext(os.path.splitext(fname)[0])[0]}/filtered'
    os.makedirs(odir, exist_ok=True)
    common.execCmd(f'/content/MyApp/FastQC/fastqc -o {odir} {file} &')


# Mapping to Reference Genome

In [ ]:
#@title Install Mapping Software
apps.install('samtools')
#apps.install('bwa')
%cd /content/temp
!wget https://jaist.dl.sourceforge.net/project/bio-bwa/bwa-0.7.17.tar.bz2
!tar xvf ./bwa-0.7.17.tar.bz2
!sed -i 's/const uint8_t rle_auxtab/extern const uint8_t rle_auxtab/g' /content/temp/bwa-0.7.17/rle.h
%cd bwa-0.7.17
!make -j16
!cp bwa /content/MyApp
%cd /content
!rm -r /content/temp/*
apps.install('picard')
apps.install('gatk')

In [ ]:
#@title Download Nematode Genome
%cd /content/Reference
!wget https://downloads.wormbase.org/species/c_elegans/PRJNA13758/sequence/genomic/c_elegans.PRJNA13758.current.genomic.fa.gz
!gunzip c_elegans.PRJNA13758.current.genomic.fa.gz

In [ ]:
#@title Mapping to Reference
import os
import glob
reference_genome = '/content/Reference/c_elegans.PRJNA13758.current.genomic.fa'
reference_label = 'worms'

os.chdir('/content/Reference')
if not os.path.exists(f'{reference_genome}.fai'):
  common.execCmd(f'samtools faidx {reference_genome}')
if not os.path.exists(f'/content/Reference/{reference_label}.pac'):
  common.execCmd(f'/content/MyApp/bwa index -p {reference_label} {reference_genome}', verbose=True)
if not os.path.exists(f'{os.path.splitext(reference_genome)[0]}.dict'):
  common.execCmd(f'''/content/MyApp/gatk/gatk CreateSequenceDictionary -R {reference_genome} -O {os.path.splitext(os.path.basename(reference_genome))[0]}.dict''', verbose=True)

for sample in samples:
  os.makedirs(f'/content/Output/{sample}', exist_ok=True)
  inputs = glob.glob(f'/content/Sample/{sample}*.filter.*')
  inputs.sort()
  common.execCmd(f'''/content/MyApp/bwa mem -t 16 -Y -M \
  -R "@RG\\tID:lib_{sample}\\tSM:{sample}\\tPL:illumina" \
  {reference_label} {inputs[0]} {inputs[1]} \
  > /content/Output/{sample}/aligned.sam''', verbose=True)
os.chdir('/content')

In [ ]:
#@title Create BAM File
import glob
import os

intermediates = []

for sample in samples:
  common.execCmd(f'''samtools view -@ 16 -b \
  -o /content/Output/{sample}/raw.bam \
  /content/Output/{sample}/aligned.sam''', verbose=True)

  common.execCmd(f'''samtools sort -l1 -T /content/temp \
  -@ 16 -O bam -o /content/Output/{sample}/sorted.bam \
  /content/Output/{sample}/raw.bam''', verbose=True)

  common.execCmd(f'''java -jar /content/MyApp/picard.jar MarkDuplicates \
  -I /content/Output/{sample}/sorted.bam \
  -O /content/Output/{sample}/{sample}.bam \
  -M /content/Output/{sample}/{sample}.metric.txt''', verbose=True)

  common.execCmd(f'samtools index /content/Output/{sample}/{sample}.bam', verbose=True)

  intermediates.append(f'/content/Output/{sample}/aligned.sam')
  intermediates.append(f'/content/Output/{sample}/raw.bam')
  intermediates.append(f'/content/Output/{sample}/sorted.bam')

for im in intermediates:
  os.system(f'rm {im}')
os.system(f'rm /content/Sample/*')

# Variant Calling

In [ ]:
#@title Install and Download Required Software and Databases
!pip install dash-bio
apps.install('sift')
# SIFT Database
!curl -L -o /content/Reference/WBcel235.83.zip "https://sift.bii.a-star.edu.sg/sift4g/public//Caenorhabditis_elegans/WBcel235.83.zip"
%cd /content/Reference
!unzip /content/Reference/WBcel235.83.zip

# SV detection
%cd /content/temp/
!curl -L -o slib4colab.tar.gz "https://firebasestorage.googleapis.com/v0/b/publicstorage-3ef6a.appspot.com/o/slib4colab.tar.gz?alt=media&token=b2f15d7c-7c37-4a3d-8613-2e0c0389b086"
!tar -xzf slib4colab.tar.gz
!bash ./slib/installOnColab.sh
!git clone https://github.com/YujiSue/Sutoku.git
!git clone https://github.com/YujiSue/Moirei.git
%cd /content/temp/Sutoku
!mkdir -p build
%cd /content/temp/Sutoku/build
!cmake ..
!make -j16
!cp -n ./sutoku /content/MyApp
%cd /content/temp/Moirei
!mkdir -p build
%cd /content/temp/Moirei/build
!cmake ..
!make -j16
!cp -n ./moirei /content/MyApp
%cd /content/temp/Moirei/plugin/mutagen
!mkdir -p build
%cd /content/temp/Moirei/plugin/mutagen/build
!cmake ..
!make
!mv ./mutagenfilter.plugin /content/MyApp
%cd /content/temp/Moirei/plugin/varhist
!mkdir -p build
%cd /content/temp/Moirei/plugin/varhist/build
!cmake ..
!make
!mv ./varhistio.plugin /content/MyApp

#
!curl -L -o /content/Reference/nematode.bin "https://firebasestorage.googleapis.com/v0/b/publicstorage-3ef6a.appspot.com/o/nematode.bin?alt=media&token=b79539da-980d-492d-9cf6-281e2079dac4"
!curl -L -o /content/Reference/nematode.db "https://firebasestorage.googleapis.com/v0/b/publicstorage-3ef6a.appspot.com/o/nematode.db?alt=media&token=e72cfaf0-601b-4cd4-93a2-99917e988f80"
# Display chart
!curl -L -o /content/Reference/nematode_circref.json "https://firebasestorage.googleapis.com/v0/b/publicstorage-3ef6a.appspot.com/o/nematode_circref.json?alt=media&token=123ef59b-6969-4a8e-a0f9-fd68eda7313f"
%cd /content

In [ ]:
#@title Variant call
for sample in samples:
  common.execCmd(f'''/content/MyApp/gatk/gatk HaplotypeCaller \
  -R {reference_genome} \
  -I /content/Output/{sample}/{sample}.bam \
  -O /content/Output/{sample}/{sample}.gatk.g.vcf \
  -ERC GVCF -G StandardAnnotation -G AS_StandardAnnotation -G StandardHCAnnotation''', verbose=True)

  common.execCmd(f'''/content/MyApp/gatk/gatk GenotypeGVCFs \
  -R {reference_genome} \
  -V /content/Output/{sample}/{sample}.gatk.g.vcf \
  -O /content/Output/{sample}/{sample}.gatk.vcf''', verbose=True)


In [ ]:
#@title Assess Variant Effect
for sample in samples:
  common.execCmd(f'''java -jar /content/MyApp/SIFT4G_Annotator.jar -c \
  -i /content/Output/{sample}/{sample}.gatk.vcf \
  -d /content/Reference/WBcel235.83 \
  -r /content/Output/{sample} -t''', verbose=True)


In [ ]:
#@title Structural Variant Detection Parameter Setup
!mkdir -p /content/Preference
common.execCmd(f'''/content/MyApp/sutoku template --outdir /content/Preference''', verbose=True)

import json
param = json.load(open('/content/Preference/param.json'))
param['variant-param']['homo'] = True
param['paired'] = paired_seq
param['depth-bin'] = 4
param['detect-sv'] = True
param['thread'] = 16
json.dump(param, open('/content/Preference/ce_svdetect.json', 'w'))


In [ ]:
#@title Create Control Data for Structural Variant Detection

#@markdown If you have sequenced the parental strain of the mutant sample, this analysis can create control data to filter out background mutations.<br>
#@markdown Otherwise, please skip this cell and run the one below to find mutations directly.
common.execCmd(f'''/content/MyApp/sutoku summary \
--bam /content/Output/{sample}/{sample}.bam \
--reference /content/Reference/nematode.bin \
--param /content/Preference/ce_svdetect.json''', verbose=True)

common.execCmd(f'''mv /content/Output/{sample}/{sample}.bam.bsm /content/Output/control.bsm''')
if os.path.exists('/content/Output/control.bsm'):
  print(f'Control data : /content/Output/control.bsm')


In [ ]:
#@title Structural Variant Detection
#@markdown Check this box if you are using control data to filter out background mutations.<br>use_control = False #@param {type: 'boolean'}<br>
use_control = False #@param {type: 'boolean'}
#@markdown *Note: The detection of a full-length mitochondrial Duplication when not using control data is a false positive due to the circular structure of the genome, which can be interpreted as a tandem duplication.
for sample in samples:
  common.execCmd(f'''/content/MyApp/sutoku analyze \
  --bam /content/Output/{sample}/{sample}.bam \
  --reference /content/Reference/nematode.bin \
  {"--control /content/Output/control.bsm" if use_control else ""}\
  --param /content/Preference/ce_svdetect.json \
  {"--paired" if paired_seq else ""} \
  --depth-bin 4 --detect-sv --thread 16 --oformat tsv''', verbose=True)
#@markdown The result file will be output as 'Output' > [Sample Name] > [Sample Name].bam.variants.tsv. The extension is (.tsv), but it can be opened with Excel.<br>
#@markdown *If Colab crashes during execution due to RAM failure, stop structural variant detection and run from the code cell below.

In [ ]:
#@title Narrowing Down Candidate Causal Genes
#@markdown - Filtering for selection of the EMS mutation pattern (G-C=>A-T) and homozygous variants.<br>
#@markdown - Variants that do not cause amino acid substitutions are deliberately kept, as they can be used to narrow down regions during histogram creation (described below).<br>
#@markdown - Histogram data is set to be exported. Creating a histogram of the remaining variants from the above processing will likely show a high frequency of mutations in a limited region of a specific chromosome. Candidate genes are often located within this high-frequency mutation region, so use this for refinement.<br>
#@markdown - The result file will be output as 'Output' > [Sample Name] > [Sample Name].filtered.tsv. The extension is (.tsv), but it can be opened with Excel.

caller = 'gatk' #@param {type: 'string'}
predictor = 'sift' #@param {type: 'string'}
ems_mut = True #@param {type: 'boolean'}
enu_mut = False #@param {type: 'boolean'}
homo_only = True #@param {type: 'boolean'}
oformat = 'tsv' #@param {type: 'string'}
export_histogram = True #@param {type: 'boolean'}
histogram_bin = '500K' #@param {type: 'string'}
for sample in samples:
  cmd = f'''
  /content/MyApp/moirei variantFilter \
  --reference /content/Reference/nematode.bin \
  --param /content/Preference/param.json \
  {"--homo-only" if homo_only else ""} \
  --plugin-filter /content/MyApp/mutagenfilter.plugin?mutagen={"EMS" if ems_mut else ("ENU" if enu_mut else "")}\
  --plugin-io /content/MyApp/varhistio.plugin?bin={histogram_bin} \
  --oformat {oformat} \
  /content/Output/{sample}/{sample}.gatk_SIFTpredictions.vcf
  '''
  common.execCmd(cmd, verbose=True)
  os.system(f"mv /content/Output/{sample}/{sample}.gatk_SIFTpredictions.filtered.tsv /content/Output/{sample}/{sample}.filtered.tsv")
  os.system(f"mv /content/Output/{sample}/{sample}.gatk_SIFTpredictions.hist.json /content/Output/{sample}/{sample}.hist.json")


In [ ]:
#@title Visualization of Variant Distribution
import json
from dash import Dash, html, dcc, Input, Output, State, callback
import dash_bio as dashbio
app = Dash(__name__)
refdata = json.load(open('/content/Reference/nematode_circref.json'))
for sample in samples:
  hist = os.path.join('/content/Output', sample, f'{sample}.hist.json')
  if not os.path.exists(hist):
    continue
  data = json.load(open(hist))
  for val in data:
    val['pos'] = str(val['start']) + '-' + str(val['end'])
  layout_config = {
      "labels": {
          "size": 24,
          "color": "#ffffff",
      },
      "ticks": {"display": False},
  }
  tracks_config = {
      "innerRadius": 300,
      "outerRadius": 400,
      "color": "OrRd",
      'tooltipContent': {'name': 'pos'},
  }
  app.layout = html.Div([
      dashbio.Circos(
          layout = refdata,
          config = layout_config,
          tracks = [
              {
                  "type": "HISTOGRAM",
                  "data": data,
                  "config": tracks_config,
              }
          ],
      )
  ])

if __name__ == "__main__":
  app.run(debug=True)

In [ ]:
#@title Compress Results and Copy to Google Drive
drive_out_directory = '/content/drive/MyDrive/result' #@param {type:'string'}
import os
os.makedirs(drive_out_directory, exist_ok=True)
for sample in samples:
  common.execCmd(f"tar -cvzf {os.path.join('/content/Output', sample + '.tar.gz')} {os.path.join('/content/Output', sample)}")
  res = common.execCmd(f"cp {os.path.join('/content/Output', sample + '.tar.gz')} {drive_out_directory}")